<a href="https://colab.research.google.com/github/josemoti1999/google_landmark_retreival_kaggle/blob/master/efficientnet5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCBRIDE SCRIPT Experiment 18

In [ ]:
%tensorflow_version 2.x
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
colab = 1
FOLDER = 'Experiment_18'

In [ ]:
!pip install -q efficientnet
!pip install -q gcsfs
import os
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from tqdm.notebook import tqdm as tqdm

In [ ]:
!pip install -q tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0
import requests
resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
if resp.status_code != 200:
  print("Failed to switch the TPU to TF {}".format(version))

In [ ]:
PATH = '/content/drive/My Drive/Kaggle/landmark_recognition'
PATH = os.path.join(PATH, FOLDER)
os.makedirs(PATH, exist_ok=True)
PATH

'/content/drive/My Drive/Kaggle/landmark_recognition/Experiment_18'

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

Running on TPU  grpc://10.83.147.250:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.83.147.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.83.147.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
GCS_PATH = 'gs://kds-4c1ac9004efd00b8a8bcfa9b3dcc800dbdf8978d612363a73d982469'
GCS_PATH_2 = 'gs://kds-7850820fffb4d34978de60497a6b764639bd2c8ff0405571fcdb6582'
DICT_PATH = 'gs://kds-a322da0692cb29e85f28040e32919375696ed752764db14e715ded3f/train_encoded.csv'

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [384, 384]
SEED = 100
LR = 3e-5
NUMBER_OF_CLASSES = 81313

In [ ]:
FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec') + tf.io.gfile.glob(GCS_PATH_2 + '/train*.tfrec')
df = pd.read_csv(DICT_PATH)
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.02, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
n_trn_classes = df[df['group'].isin(training_groups)]['landmark_id_encode'].nunique()
n_val_classes = df[df['group'].isin(validation_groups)]['landmark_id_encode'].nunique()
print('Validation tfrecord-',validation_groups)
print(f'The number of unique training classes is {n_trn_classes} of {NUMBER_OF_CLASSES} total classes')
print(f'The number of unique validation classes is {n_val_classes} of {NUMBER_OF_CLASSES} total classes')

Validation tfrecord- [14]
The number of unique training classes is 81313 of 81313 total classes
The number of unique validation classes is 24574 of 81313 total classes


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.random_flip_left_right(image)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    #temp = tf.unstack(image, axis=-1)
    #image = tf.stack([temp[2], temp[1], temp[0]], axis=-1)
    return image

def read_tfrecord(example):
    TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    target = tf.cast(example['target'], tf.int32)
    return image, target

def load_dataset(filenames, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) 
    return dataset

def arcface_format(image, target):
    return {'inp1': image, 'inp2': target}, target

def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(filenames, ordered = True, prediction = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    if prediction:
        dataset = dataset.batch(BATCH_SIZE * 4)
    else:
        dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    df = pd.read_csv(DICT_PATH)
    n = df[df['group'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')

Training with 1548860 images
Validating with 31610 images


In [ ]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.0000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
     
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
    return lr_callback

class ArcMarginProduct(tf.keras.layers.Layer):
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):
        super(ArcMarginProduct, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])
        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
def get_model():
    with strategy.scope():
        margin = ArcMarginProduct(
            n_classes = NUMBER_OF_CLASSES, 
            s = 64, 
            m = 0.1, 
            name='head/arc_margin', 
            dtype='float32'
            )
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x0 = efn.EfficientNetB5(weights = 'noisy-student', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x0)
        x = margin([x, label])
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
        #opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
        opt = tf.keras.optimizers.SGD(learning_rate = 1e-3, momentum=0.99)
        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        return model

In [ ]:
df=df.sort_values(by='landmark_id_encode', ascending=True)
y = df['landmark_id_encode'].values
weight_map = 1/np.log(np.bincount(y))
weight_map = dict(enumerate(weight_map))
print(weight_map[55807],weight_map[50843],weight_map[44791],weight_map[6966]) 

0.11436608996297955 0.12969823432482586 1.4426950408889634 1.4426950408889634


In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# df=df.sort_values(by='landmark_id_encode', ascending=True)
# y = df['landmark_id_encode'].values
# classes = df['landmark_id_encode'].unique()
# weight_map = compute_class_weight('balanced', classes, y)
# weight_map = dict(enumerate(weight_map))
# print(weight_map[55807],weight_map[50843],weight_map[44791],weight_map[6966])   
# #0.003,0.008,9.71,9.71

In [ ]:
#change 3 explicitly
EPOCHS = 20
SCALE = 64
MARGIN = 0.1
EFNET = 5
RETRAIN = True
REDUCE_LR = True
MODEL_PATH = 'fold-0_epoch-5_valloss-1.3085_loss-0.0085_margin-0.10_scale-64_logweight.h5'
INITIAL_EPOCH = 0
WEIGHT_MAP = weight_map

In [ ]:
seed_everything(SEED)

train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

model = get_model()
if RETRAIN:
    print('Loading model...')
    model.load_weights(os.path.join(PATH, MODEL_PATH))

checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(PATH, 'fold-0_epoch-{epoch}_valloss-{val_loss:.4f}_loss-{loss:.4f}_margin-0.10_scale-64_logweight.h5'), 
                                                 monitor = 'val_loss', 
                                                 save_best_only = False, 
                                                 save_weights_only = False)
if REDUCE_LR:
    cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', 
                                                        mode = 'min', 
                                                        factor = 0.8, 
                                                        patience = 2, 
                                                        verbose = 0, 
                                                        min_delta = 0.0001)
else:
    cb_lr_schedule = get_lr_callback()
history = model.fit(train_dataset,  
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = EPOCHS,
                    initial_epoch=INITIAL_EPOCH,
                    callbacks = [cb_lr_schedule, checkpoint],
                    validation_data = val_dataset,
                    class_weight = WEIGHT_MAP,
                    verbose = 1)

115261440/115255328 [==============================] - 1s 0us/step
Loading model...
Epoch 1/20
12100/12100 [==============================] - 6426s 531ms/step - loss: 0.0083 - sparse_categorical_accuracy: 0.9964 - val_loss: 1.3121 - val_sparse_categorical_accuracy: 0.8065 - lr: 0.0010
Epoch 2/20
 7324/12100 [=================>............] - ETA: 41:20 - loss: 0.0084 - sparse_categorical_accuracy: 0.9964